In [8]:
!pip install scikit-fuzzy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip


In [13]:
import sys
!{sys.executable} -m pip install scikit-fuzzy

     -------------------------------------- 920.8/920.8 kB 5.3 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl


In [15]:

def define_fuzzy_variables():
    """
    Define the fuzzy variables (inputs and output) and their membership functions.
    """
    # Input: Soil Moisture (0-100%)
    soil_moisture = ctrl.Antecedent(np.arange(0, 101, 1), 'soil_moisture')
    soil_moisture['Dry'] = fuzz.trapmf(soil_moisture.universe, [0, 0, 20, 40])
    soil_moisture['Moist'] = fuzz.trimf(soil_moisture.universe, [30, 50, 70])
    soil_moisture['Wet'] = fuzz.trapmf(soil_moisture.universe, [60, 80, 100, 100])

    # Input: Temperature (0-50°C)
    temperature = ctrl.Antecedent(np.arange(0, 51, 1), 'temperature')
    temperature['Low'] = fuzz.trapmf(temperature.universe, [0, 0, 10, 20])
    temperature['Medium'] = fuzz.trimf(temperature.universe, [15, 25, 35])
    temperature['High'] = fuzz.trapmf(temperature.universe, [30, 40, 50, 50])

    # Output: Water Flow (0-100%)
    water_flow = ctrl.Consequent(np.arange(0, 101, 1), 'water_flow')
    water_flow['Low'] = fuzz.trapmf(water_flow.universe, [0, 0, 20, 40])
    water_flow['Medium'] = fuzz.trimf(water_flow.universe, [30, 50, 70])
    water_flow['High'] = fuzz.trapmf(water_flow.universe, [60, 80, 100, 100])

    return soil_moisture, temperature, water_flow

In [16]:

def define_fuzzy_rules(soil_moisture, temperature, water_flow):
    """
    Define the fuzzy rules for the system.
    """
    rules = [
        ctrl.Rule(soil_moisture['Dry'] & temperature['High'], water_flow['High']),
        ctrl.Rule(soil_moisture['Dry'] & temperature['Medium'], water_flow['Medium']),
        ctrl.Rule(soil_moisture['Dry'] & temperature['Low'], water_flow['Medium']),
        ctrl.Rule(soil_moisture['Moist'] & temperature['High'], water_flow['Medium']),
        ctrl.Rule(soil_moisture['Moist'] & temperature['Medium'], water_flow['Low']),
        ctrl.Rule(soil_moisture['Moist'] & temperature['Low'], water_flow['Low']),
        ctrl.Rule(soil_moisture['Wet'] & temperature['High'], water_flow['Low']),
        ctrl.Rule(soil_moisture['Wet'] & temperature['Medium'], water_flow['Low']),
        ctrl.Rule(soil_moisture['Wet'] & temperature['Low'], water_flow['Low']),
    ]
    return rules


In [17]:

def create_fuzzy_system(rules):
    """
    Create and return the fuzzy control system and simulation object.
    """
    irrigation_ctrl = ctrl.ControlSystem(rules)
    return ctrl.ControlSystemSimulation(irrigation_ctrl)



In [18]:

def main():
    # Define fuzzy variables
    soil_moisture, temperature, water_flow = define_fuzzy_variables()

    # Define fuzzy rules
    rules = define_fuzzy_rules(soil_moisture, temperature, water_flow)

    # Create fuzzy system simulation
    irrigation_sim = create_fuzzy_system(rules)

    # Get user input
    try:
        soil_moisture_input = float(input("Enter the soil moisture level (0-100%): "))
        temperature_input = float(input("Enter the temperature (°C) (0-50): "))

        # Validate inputs
        if not (0 <= soil_moisture_input <= 100):
            raise ValueError("Soil moisture must be between 0 and 100.")
        if not (0 <= temperature_input <= 50):
            raise ValueError("Temperature must be between 0 and 50.")

        # Set inputs
        irrigation_sim.input['soil_moisture'] = soil_moisture_input
        irrigation_sim.input['temperature'] = temperature_input

        # Compute output
        irrigation_sim.compute()

        # Display results
        print(f"\nSoil Moisture Input: {soil_moisture_input}%")
        print(f"Temperature Input: {temperature_input}°C")
        print(f"Recommended Water Flow: {irrigation_sim.output['water_flow']:.2f}%")

    except ValueError as e:
        print(f"Error: {e}")


if __name__ == "__main__":
    main()
    


Soil Moisture Input: 50.0%
Temperature Input: 45.0°C
Recommended Water Flow: 50.00%
